In [15]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [16]:
n = lk('../lookup_tables/nfet_03v3.mat')

#### Specifications

In [17]:
gm = 10e-3

#### Design choices

In [18]:
l = 0.28
gm_id = np.array([5, 10, 20])

#### Sizing and benchmarking

In [19]:
id = gm/gm_id
jd = n.lookup('ID_W', GM_ID=gm_id, L=l)
w = id/jd

cgg_w = n.lookup('CGG_W', GM_ID=gm_id, L=l)
cgg = w*cgg_w
ft = gm/cgg/2/np.pi

df = pd.DataFrame([gm_id, id, jd, w, cgg, ft], ['gm_id', 'id', 'jd', 'w', 'cgg', 'ft'], columns=['option1', 'option2', 'option3']); df

,option1,option2,option3
gm_id,5.000000e+00,1.000000e+01,2.000000e+01
id,2.000000e-03,1.000000e-03,5.000000e-04
jd,2.539270e-05,6.847242e-06,4.903808e-07
w,7.876281e+01,1.460442e+02,1.019616e+03
cgg,9.257212e-14,1.603948e-13,9.581458e-13
ft,1.719253e+10,9.922699e+09,1.661072e+09


#### Spice validation

In [ ]:
%%writefile ./sizing_diffpair.spice
** differential pair sizing example

.lib /foss/pdks/gf180mcuD/libs.tech/ngspice/sm141064.ngspice typical
.inc /foss/pdks/gf180mcuD/libs.tech/ngspice/design.ngspice
.param lx=0.28e-6 wx=146e-6 nfx=30, idx=1e-3

XM1a d g s 0 nfet_03v3 L={lx} W={wx} nf=30 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
XM1b d g s 0 nfet_03v3 L={lx} W={wx} nf=30 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
vg  g  0  1
vd  d  0  2
is  s  0  {2*idx}

.control
  op
  show
  save @m.xM1a.m0[gm]
  save @m.xM1a.m0[cgg]
  save @m.xM1a.m0[cgso]
  save @m.xM1a.m0[cgdo]

  print @m.xM1a.m0[gm]
  print @m.xM1a.m0[cgg]
  print @m.xM1a.m0[cgso]
  print @m.xM1a.m0[cgdo]
    
.endc
.end

Overwriting ./sizing_diffpair.spice


In [25]:
!PDK=gf180mcuD /foss/tools/bin/ngspice -b ./sizing_diffpair.spice


Note: Compatibility modes selected: hs a



Circuit: ** differential pair sizing example

Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver

No. of Data Rows : 1
 BSIM4v5: Berkeley Short Channel IGFET Model-4
     device             m.xm1b.m0             m.xm1a.m0
      model          nfet_03v3.12          nfet_03v3.12
          l               2.8e-07               2.8e-07
          w               0.00146               0.00146
          m                     1                     1
         nf                    30                    30
         sa                     0                     0
         sb                     0                     0
         sd                     0                     0
        sca                     0                     0
        scb                     0                     0
        scc                     0                     0
         sc                     0                     0
        min   